This notebook will be mainly used for the capstone project.

### Install geopy to find latitudes and longitudes.

In [2]:
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.52         |     pyhd8ed1ab_0          35 KB  conda-forge
    geopy-2.2.0                |     pyhd8ed1ab_0          67 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         102 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.52-pyhd8ed1ab_0
  geopy              conda-forge/noarch::geopy-2.2.0-pyhd8ed1ab_0



geographiclib-1.52   | 35 KB     | ##################################### | 100% 
geopy-2.2.0          | 67 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transa

### Install beautiful soup to web scrape a page that shows all of the venue categories and ids.

In [3]:
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.3       |     pyhb0f4dca_0          86 KB  conda-forge
    soupsieve-2.0.1            |             py_1          30 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         116 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     conda-forge/noarch::beautifulsoup4-4.9.3-pyhb0f4dca_0
  soupsieve          conda-forge/noarch::soupsieve-2.0.1-py_1



beautifulsoup4-4.9.3 | 86 KB     | ##################################### | 100% 
soupsieve-2.0.1      | 30 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing

### Import relevant packages. 

In [4]:
import pandas as pd #Import pandas to use data frames. 
import requests #Import requests to handle data from Foursquare as JSON files.
from pandas import json_normalize #Import json_normalize to easily convert json file into a pandas dataframe. 
from sklearn.cluster import KMeans #Import KMeans from sklearn to put data points into clusters.
from geopy.geocoders import Nominatim #Import Nominatim to find latitude and longitude of cities.
import numpy as np #Import numpy to do statistical calculations of vectors and to create vectors. 
import folium #Import folium to create maps.
import matplotlib.cm as cm #Import Matplotlib's cm to create rainbow vector.
import matplotlib.colors as colors #Import Matplotlib's colors to get colors from rainbow vector.
from bs4 import BeautifulSoup #Import Beautiful Soup to do web scraping. 

### My fourquare access information.

In [5]:
CLIENT_ID = 'CPKUF5JGHYZMWBIX1BMCK3QM30NIHWJ2IDCQHN1CRBX4UW2G' #My Foursquare ID.
CLIENT_SECRET = 'A5M21YUKPT00LNTFW42W0SNORNWXQEBGCH0EK31SRGEEIALF' #My Foursquare secret.
ACCESS_TOKEN = '1S4LQ2H4DTD411ZHPV3QWDMNPGKNPX2OWH4AL1QZJZHCYSSF' #My Foursquare access token.
VERSION = '20180604' #My Foursquare version.
LIMIT = 50 #The limit on the number of venues I want for each call. 

### Create a beautiful soup object to easily access category data.

In [7]:
url = "https://developer.foursquare.com/docs/build-with-foursquare/categories/" #Webpage that has venue categories.
data  = requests.get(url).text #Get HTML text from webpage.
soup = BeautifulSoup(data,"html5lib")  #Create a soup object using the variable 'data'.

### Create a function that converts the beautiful object into a pandas data frame. 

In [27]:
def create_table(data):
    categories = []
    ids = []
    def add(ul):
        categories.append(ul.h3.text)
        ids.append(ul.p.text)
        if ul.ul.h3 is not None:
            for ulul in ul.ul.contents:
                add(ulul)
    for d in data.contents:
        add(d)
    return pd.DataFrame({"ID": ids}, index = categories).sort_index()

### Create the pandas data frame to get category data.

In [28]:
cat_dict = create_table(soup.find("ul", class_ = "VenueCategories__Wrapper-sc-1ysxg0y-0 dikXMT"))

### Optional: Save pandas dataframe as a csv to easily see through all of the different venue categories.

In [29]:
cat_dict.to_csv("Categories.csv")

### Here is a look at some of the rows of the table.

In [30]:
cat_dict.head(20)

,ID
ATM,52f2ab2ebcbc57f1066b8b56
Abruzzo Restaurant,55a5a1ebe4b013909087cbb6
Acai House,5294c7523cf9994f4e043a62
Accessories Store,4bf58dd8d48988d102951735
Acehnese Restaurant,52960eda3cf9994f4e043ac9
Acupuncturist,52e81612bcbc57f1066b7a3b
Adult Boutique,5267e446e4b0ec79466e48c4
Adult Education Center,56aa371ce4b08b9a8d573570
Advertising Agency,52e81612bcbc57f1066b7a3d
Afghan Restaurant,503288ae91d4c4b30a586d67


### Create a function that takes a list of categories and a city and outputs a table of venues from that city of those categories.

In [34]:
def venues(categories, city):
    ids = ",".join([cat_dict.at[x, "ID"] for x in categories])
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&near={}&oauth_token={}&v={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, city, ACCESS_TOKEN, VERSION, LIMIT, ids)
    results = requests.get(url).json()
    venues = results['response']['venues']
    df = pd.json_normalize(venues)

    def clean_df(df):
        filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
        new_df = df.loc[:, filtered_columns]

        def get_category_type(row):
            try:
                categories_list = row['categories']
            except:
                categories_list = row['venue.categories']

            if len(categories_list) == 0:
                return None
            else:
                return categories_list[0]['name']

        new_df['categories'] = new_df.apply(get_category_type, axis=1)
        new_df.columns = [column.split('.')[-1] for column in new_df.columns]
        new_df = new_df[new_df['categories'].isin(categories)]
        return new_df.sort_values(by = "categories")

    return clean_df(df).sort_values(by = "categories").reset_index(drop = True)

### Suppose I wanted to create a table of all of the venues that are yoga studios and casinos from Bozeman, MT. Then, I would use the "venues" function in the following way. 

In [38]:
categories = ["Yoga Studio", "Casino"]
bozeman_df = venues(categories, "Bozeman, MT")
bozeman_df

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,postalCode,cc,city,state,country,formattedAddress,id
0,Rhino Casino,Casino,412 W Main St,NaN,45.778753,-111.182783,"[{'label': 'display', 'lat': 45.778753, 'lng':...",59714,US,Belgrade,MT,United States,"[412 W Main St, Belgrade, MT 59714]",4c20e6cd0110b713f8a49ae7
1,The Angus Steakhouse: Lounge & Casino,Casino,NaN,NaN,45.679884,-111.044994,"[{'label': 'display', 'lat': 45.67988447642763...",59715,US,Bozeman,MT,United States,"[Bozeman, MT 59715]",4ca91c4714c33704c51edb3b
2,Magic Diamond,Casino,NaN,NaN,45.673191,-111.186785,"[{'label': 'display', 'lat': 45.673191, 'lng':...",59718,US,Bozeman,MT,United States,"[Bozeman, MT 59718]",6015e0832b430e6e1ffd00ec
3,Lucky Lil's,Casino,1510 N 19th Ave,NaN,45.695055,-111.061805,"[{'label': 'display', 'lat': 45.695055, 'lng':...",59718,US,Bozeman,MT,United States,"[1510 N 19th Ave, Bozeman, MT 59718]",51e5fd0e498e72c2cae20646
4,Village Inn Pizza Game Room,Casino,806 N 7th Ave,NaN,45.687841,-111.045819,"[{'label': 'display', 'lat': 45.687841, 'lng':...",59715,US,Bozeman,MT,United States,"[806 N 7th Ave, Bozeman, MT 59715]",4e279fb7d164bfc24f1b0fd6
5,Montana Lil's,Casino,29001 Norris Rd,NaN,45.672428,-111.186378,"[{'label': 'entrance', 'lat': 45.671559, 'lng'...",59718,US,Bozeman,MT,United States,"[29001 Norris Rd, Bozeman, MT 59718]",51e5fd0e498e30815ba0ac6f
6,Lucky Lils Casino,Casino,2401 W Main St,NaN,45.676901,-111.069582,"[{'label': 'display', 'lat': 45.676901, 'lng':...",59718,US,Bozeman,MT,United States,"[2401 W Main St, Bozeman, MT 59718]",4ddb2d65e4cd1a49b7d89ae9
7,Montana Lil's,Casino,629 1/2 W 1st St,NaN,45.765337,-111.189458,"[{'label': 'display', 'lat': 45.76533740210031...",59501,US,Havre,MT,United States,"[629 1/2 W 1st St, Havre, MT 59501]",51e5fd1b498e74446c73191f
8,Montana Lil's,Casino,110 West Main,NaN,45.856667,-111.333455,"[{'label': 'display', 'lat': 45.8566673, 'lng'...",59741,US,Manhattan,MT,United States,"[110 West Main, Manhattan, MT 59741]",51e5fd25498eec1a5f12602b
9,Lucky Lil's,Casino,206B Jefferson Ave,NaN,45.771453,-111.184327,"[{'label': 'display', 'lat': 45.771453, 'lng':...",59714,US,Belgrade,MT,United States,"[206B Jefferson Ave, Belgrade, MT 59714]",4f32c8ac19836c91c7fa39f9


In [39]:
categories = ["Yoga Studio", "Casino"]
sidney_df = venues(categories, "Sidney, MT")
sidney_df

,name,categories,address,lat,lng,labeledLatLngs,postalCode,cc,city,state,country,formattedAddress,id
0,Montana Lil's,Casino,1101 S Central Ave,47.706232,-104.163481,"[{'label': 'display', 'lat': 47.7062315, 'lng'...",59270,US,Sidney,MT,United States,"[1101 S Central Ave, Sidney, MT 59270]",4f32b84419836c91c7f30b20
1,Lucky Lil's,Casino,1157 S Central Ave,47.705729,-104.166456,"[{'label': 'display', 'lat': 47.705729, 'lng':...",59270,US,Sidney,MT,United States,"[1157 S Central Ave, Sidney, MT 59270]",51e5fd2f498ec67b12f033f4


### As someone who loves yoga but hates gambling, it seems that Bozeman is more desirable for me than Sidney. Yes, it is true that Bozeman has more casinos, but it has a higher yoga to casino ratio, since Sidney does not have any yoga studios. I am concerned mostly with relative percentages when comparing different cities. 

### The following is a list of cities that I am open to moving to. The cities are from the states of Montana, Idaho, North Dakota, South Dakota, Wyoming, Nebraska, and Iowa. I want to keep this list for later to look at their venue category percentages for various category lists. 

In [40]:
cities = ["Billings, MT", "Missoula, MT", "Great Falls, MT", "Bozeman, MT", "Boise, ID", "Meridian, ID", "Nampa, ID", "Coeur d'Alene, ID", "Fargo, ND", "Sioux Falls, SD", "Rapid City, SD", "Lincoln, NE", "Omaha, NE", "Bellevue, NE", "Des Moines, IA", "Cedar Rapids, IA", "Cheyenne, WY", "Casper, WY", "Laramie, WY"]

### Here is a function that takes a list of cities and list of categories and outputs, for each city, the percentage of venues of each category in the category list. 

In [41]:
def venue_percentages(categories, cities):
    def perc(dic, key, denom):
        value = dic.get(key)
        if value is None:
            return 0
        else:
            return value / denom
    data = {}
    for city in cities:
        city_df = venues(categories, city)
        venue_count = city_df.groupby('categories').count()['name'].to_dict()
        venue_perc = dict([(category, perc(venue_count, category, city_df.shape[0])) for category in categories])
        data[city] = venue_perc
    df = pd.DataFrame(data).T
    for category in categories:
        if sum(df[category]) == 0:
            del df[category]
    return df

### To cluster cities, I am going to use three sports categories, three performing arts categories, and three outdoor categories. 

In [47]:
sports_categories = ["Baseball Stadium", "Basketball Stadium", "Football Stadium"]
performing_arts_categories = ["Theater", "Dance Studio", "Concert Hall"]
outdoor_categories = ["Lake", "Mountain", "River"]
categories = sports_categories + performing_arts_categories + outdoor_categories

### I will now create a data frame that gives the venue category percentages for my cities.

In [49]:
df = venue_percentages(categories, cities)

In [50]:
df

,Baseball Stadium,Basketball Stadium,Football Stadium,Theater,Dance Studio,Concert Hall,Lake,Mountain,River
"Billings, MT",0.050000,0.000000,0.000000,0.350000,0.200000,0.150000,0.050000,0.000000,0.200000
"Missoula, MT",0.048780,0.024390,0.024390,0.146341,0.195122,0.073171,0.097561,0.121951,0.268293
"Great Falls, MT",0.062500,0.062500,0.062500,0.062500,0.000000,0.125000,0.125000,0.250000,0.250000
"Bozeman, MT",0.000000,0.000000,0.025641,0.153846,0.179487,0.051282,0.179487,0.256410,0.153846
"Boise, ID",0.073171,0.024390,0.000000,0.097561,0.121951,0.097561,0.341463,0.097561,0.146341
"Meridian, ID",0.097561,0.048780,0.000000,0.097561,0.146341,0.073171,0.317073,0.097561,0.121951
"Nampa, ID",0.100000,0.050000,0.000000,0.100000,0.150000,0.125000,0.225000,0.075000,0.175000
"Coeur d'Alene, ID",0.000000,0.000000,0.000000,0.025000,0.075000,0.000000,0.725000,0.075000,0.100000
"Fargo, ND",0.051282,0.051282,0.076923,0.153846,0.179487,0.076923,0.205128,0.025641,0.179487
"Sioux Falls, SD",0.128205,0.000000,0.025641,0.153846,0.153846,0.179487,0.205128,0.025641,0.128205


### Here is my function to put each city in a cluster based off the previous data. 

In [11]:
def add_clusters(df, clusters):
    new_df = df
    k_means = KMeans(init="k-means++", n_clusters=clusters, n_init=12)
    k_means.fit(new_df)
    new_df["Labels"] = k_means.labels_
    return new_df.sort_values("Cluster Labels")

In [12]:
df = add_clusters(df, 5)

In [13]:
df

,Gym,Yoga Studio,Pilates Studio,Country Dance Club,Dance Studio,Salsa Club,Skating Rink,Lake,Concert Hall,Hot Spring,Labels
"Cedar Rapids, IA",0.333333,0.000000,0.000000,0.00000,0.030303,0.000,0.060606,0.454545,0.060606,0.060606,0
"Des Moines, IA",0.375000,0.062500,0.000000,0.03125,0.093750,0.000,0.031250,0.375000,0.000000,0.031250,0
"Bellevue, NE",0.406250,0.031250,0.000000,0.00000,0.031250,0.000,0.062500,0.468750,0.000000,0.000000,0
"Omaha, NE",0.393939,0.030303,0.000000,0.00000,0.030303,0.000,0.060606,0.484848,0.000000,0.000000,0
"Billings, MT",0.666667,0.060606,0.030303,0.00000,0.090909,0.000,0.030303,0.030303,0.090909,0.000000,1
"Fargo, ND",0.553191,0.021277,0.000000,0.00000,0.085106,0.000,0.148936,0.127660,0.042553,0.021277,1
"Sioux Falls, SD",0.589744,0.102564,0.000000,0.00000,0.076923,0.000,0.025641,0.153846,0.051282,0.000000,1
"Meridian, ID",0.538462,0.102564,0.000000,0.00000,0.051282,0.000,0.000000,0.230769,0.051282,0.025641,1
"Boise, ID",0.525000,0.100000,0.000000,0.00000,0.050000,0.000,0.000000,0.250000,0.050000,0.025000,1
"Great Falls, MT",0.700000,0.100000,0.000000,0.00000,0.000000,0.000,0.000000,0.100000,0.100000,0.000000,1


In [31]:
address = 'Lincoln, NE'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Lincoln, NE are 41.1280799, -101.1313806.


In [21]:
def add_lats_lngs(df):
    new_df = df
    geolocator = Nominatim(user_agent="explorer")
    lats = {}
    lngs = {}
    for city in cities:
        location = geolocator.geocode(city)
        latitude = location.latitude
        longitude = location.longitude
        lats[city] = latitude
        lngs[city] = longitude
    new_df["Latitude"] = pd.Series(lats)
    new_df["Longitude"] = pd.Series(lngs)
    return new_df

    

In [36]:
df = add_lats_lngs(df)

In [37]:
df

,Gym,Yoga Studio,Pilates Studio,Country Dance Club,Dance Studio,Salsa Club,Skating Rink,Lake,Concert Hall,Hot Spring,Labels,Latitude,Longitude
"Cedar Rapids, IA",0.333333,0.000000,0.000000,0.00000,0.030303,0.000,0.060606,0.454545,0.060606,0.060606,0,41.975887,-91.670405
"Des Moines, IA",0.375000,0.062500,0.000000,0.03125,0.093750,0.000,0.031250,0.375000,0.000000,0.031250,0,41.591032,-93.604665
"Bellevue, NE",0.406250,0.031250,0.000000,0.00000,0.031250,0.000,0.062500,0.468750,0.000000,0.000000,0,13.547005,2.112145
"Omaha, NE",0.393939,0.030303,0.000000,0.00000,0.030303,0.000,0.060606,0.484848,0.000000,0.000000,0,41.258746,-95.938376
"Billings, MT",0.666667,0.060606,0.030303,0.00000,0.090909,0.000,0.030303,0.030303,0.090909,0.000000,1,45.787496,-108.496070
"Fargo, ND",0.553191,0.021277,0.000000,0.00000,0.085106,0.000,0.148936,0.127660,0.042553,0.021277,1,46.877229,-96.789821
"Sioux Falls, SD",0.589744,0.102564,0.000000,0.00000,0.076923,0.000,0.025641,0.153846,0.051282,0.000000,1,43.546059,-96.731393
"Meridian, ID",0.538462,0.102564,0.000000,0.00000,0.051282,0.000,0.000000,0.230769,0.051282,0.025641,1,43.608629,-116.392326
"Boise, ID",0.525000,0.100000,0.000000,0.00000,0.050000,0.000,0.000000,0.250000,0.050000,0.025000,1,43.616616,-116.200886
"Great Falls, MT",0.700000,0.100000,0.000000,0.00000,0.000000,0.000,0.000000,0.100000,0.100000,0.000000,1,47.504885,-111.291890


In [38]:
def cluster_map(df):  
    map_clusters = folium.Map(location=[np.mean(df["Latitude"]), np.mean(df["Longitude"])], zoom_start=1)

    # set color scheme for the clusters
    kclusters = np.max(df["Labels"]+1)
    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map
    markers_colors = []
    for city, lat, lon, cluster in zip(df.index.values, df['Latitude'], df['Longitude'], df['Labels']):
        label = folium.Popup(str(city) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)

    return map_clusters

In [39]:
my_map = cluster_map(df)

In [40]:
my_map

In [46]:
df.at["Lincoln, NE", "Latitude"] = 40.813618

In [47]:
df.at["Lincoln, NE", "Longitude"] = -96.702599

In [48]:
df

,Gym,Yoga Studio,Pilates Studio,Country Dance Club,Dance Studio,Salsa Club,Skating Rink,Lake,Concert Hall,Hot Spring,Labels,Latitude,Longitude
"Cedar Rapids, IA",0.333333,0.000000,0.000000,0.00000,0.030303,0.000,0.060606,0.454545,0.060606,0.060606,0,41.975887,-91.670405
"Des Moines, IA",0.375000,0.062500,0.000000,0.03125,0.093750,0.000,0.031250,0.375000,0.000000,0.031250,0,41.591032,-93.604665
"Bellevue, NE",0.406250,0.031250,0.000000,0.00000,0.031250,0.000,0.062500,0.468750,0.000000,0.000000,0,13.547005,2.112145
"Omaha, NE",0.393939,0.030303,0.000000,0.00000,0.030303,0.000,0.060606,0.484848,0.000000,0.000000,0,41.258746,-95.938376
"Billings, MT",0.666667,0.060606,0.030303,0.00000,0.090909,0.000,0.030303,0.030303,0.090909,0.000000,1,45.787496,-108.496070
"Fargo, ND",0.553191,0.021277,0.000000,0.00000,0.085106,0.000,0.148936,0.127660,0.042553,0.021277,1,46.877229,-96.789821
"Sioux Falls, SD",0.589744,0.102564,0.000000,0.00000,0.076923,0.000,0.025641,0.153846,0.051282,0.000000,1,43.546059,-96.731393
"Meridian, ID",0.538462,0.102564,0.000000,0.00000,0.051282,0.000,0.000000,0.230769,0.051282,0.025641,1,43.608629,-116.392326
"Boise, ID",0.525000,0.100000,0.000000,0.00000,0.050000,0.000,0.000000,0.250000,0.050000,0.025000,1,43.616616,-116.200886
"Great Falls, MT",0.700000,0.100000,0.000000,0.00000,0.000000,0.000,0.000000,0.100000,0.100000,0.000000,1,47.504885,-111.291890


In [49]:
my_map = cluster_map(df)

In [50]:
my_map

In [51]:
def relative_percentages_per_group(df):
    df_gb = df.groupby('Labels').mean()
    for category in df_gb.columns:
        total = sum(df_gb[category])
        df_gb[category] = [value / total for value in df_gb[category]]
    return df_gb

In [52]:
df_gb = relative_percentages_per_group(df)

In [53]:
df_gb

,Gym,Yoga Studio,Pilates Studio,Country Dance Club,Dance Studio,Salsa Club,Skating Rink,Lake,Concert Hall,Hot Spring,Latitude,Longitude
Labels,,,,,,,,,,,,
0,0.182201,0.074784,0.000000,1.0,0.097480,0.0,0.315628,0.280109,0.091928,0.295556,0.161148,0.136837
1,0.287308,0.203095,0.291268,0.0,0.129392,0.0,0.171900,0.091630,0.401262,0.179374,0.209315,0.213610
2,0.184889,0.330277,0.708732,0.0,0.251770,0.0,0.243284,0.147165,0.355128,0.203311,0.198476,0.204498
3,0.106052,0.000000,0.000000,0.0,0.153717,0.0,0.000000,0.444442,0.000000,0.000000,0.222085,0.229020
4,0.239550,0.391843,0.000000,0.0,0.367640,1.0,0.269188,0.036654,0.151682,0.321760,0.208976,0.216035
